# Prompt Optimisation

As your system built on AgentX goes to production, whether it is using the Agent class or publishing Tool to the store, you will accumulate more and more feedback data. Utilising these data, you can tune the prompt and the inference hyperparameters to achieve elevated performance.

This notebook demonstrate how to tune prompt and the inference hyperparameters to better predict the salary range of a job posting based on its job description, which can be used to help job seekers and hiring managers to better understand the job market.

In [93]:
# Data sourced from https://www.kaggle.com/datasets/arshkon/linkedin-job-postings
# CC BY-SA 4.0 license
# Data cleaned and filtered by Chan Ka Hei

from pandas import read_parquet
from rich import print as rich_print

dataset = read_parquet(
    'data/job_posting_2023.parquet'
)

dataset.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,salary,short_description,skill_set
20487,3701151544,20122211.0,Senior Litigation Attorney/Special Assistant A...,SENIOR LITIGATION ATTORNEY/SPECIAL ASSISTANT A...,128252.0,NaN,111251.0,YEARLY,Full-time,"Providence, RI",...,1.692730e+12,None,1,FULL_TIME,USD,BASE_SALARY,1,119751.5,The Rhode Island Office of the Attorney Genera...,"[""Trial experience"", ""Supervisory experience"",..."
25611,3697340789,90639.0,Site Civil Engineer,Green Key is looking for a strong site design ...,120000.0,NaN,100000.0,YEARLY,Full-time,"New York, NY",...,1.692730e+12,None,1,FULL_TIME,USD,BASE_SALARY,1,110000.0,Green Key is seeking a Civil Engineer with exp...,"[""Autocad"", ""Civil 3D"", ""Stormwater management..."
22215,3699085089,1441.0,"Software Engineering Manager II, Google Cloud ...",Note: By applying to this position you will ha...,283000.0,NaN,185000.0,YEARLY,Full-time,"Seattle, WA",...,1.692830e+12,careers.google.com,1,FULL_TIME,USD,BASE_SALARY,1,234000.0,"Like Google's own ambitions, the work of a Sof...","[""Bachelor\u2019s degree or equivalent practic..."
821,3757776363,2908367.0,Licensed Journeyman Electician,Gpac just partnered with a super company to fi...,100000.0,NaN,70000.0,YEARLY,Full-time,"San Diego, CA",...,1.699070e+12,usa.applybe.com,0,FULL_TIME,USD,BASE_SALARY,1699138477,85000.0,Gpac has partnered with a company to find Sola...,"[""Journeyman Electrician license"", ""At least 3..."
7506,3757446699,8052981.0,Senior Buyer,"ObjectiveBroadband Telecom Power, an EV chargi...",90000.0,NaN,80000.0,YEARLY,Full-time,"Santa Ana, CA",...,1.699040e+12,None,0,FULL_TIME,USD,BASE_SALARY,1699039150,85000.0,"Broadband Telecom Power, an EV charging equipm...","[""Bachelor's Degree in Business, Engineering o..."


In [94]:
# We will use 'salary' as our target variable.
# We will use 'title', 'short_description', 'skill_set', 'location', 'formatted_experience_level' as our input variable.

# Remove curreny other than USD

x = dataset[
    [
        'title', 
        'short_description',
        'skill_set',
        'location', 
        'formatted_experience_level', 
    ]
].to_dict(orient='records')

y = dataset['salary'].to_list()

In [73]:
# Use AgentX to generate a model that can predict salary based on job title, description, location, and experience level.
# This will be the baseline of the optimisation process.

from agentx.agent import Agent
from agentx.schema import GenerationConfig, Message, Content
from dotenv import load_dotenv
from random import sample
from typing import List, Union
from pydantic import BaseModel
import os
import asyncio

load_dotenv()

class JobPost(BaseModel):
    title: str
    short_description: str
    skill_set: List[str]
    location: Union[str, None]
    formatted_experience_level: Union[str, None]

class SalaryPrediction(BaseModel):
    reasons: str
    salary: int

generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
    azure_deployment='gpt-35',
)

salary_prediction_agent = Agent(
    name='salary_prediction',
    generation_config=generation_config,
    system_prompt='''You will predict the salary of a job posting based on the job title, description, location, and experience level.
Output JSON format only.''',
)

# sample a small test set
test_set_ids = sample(range(len(x)), 50)
rich_print(x[test_set_ids[1]])
rich_print(y[test_set_ids[1]])

{
    'title': 'AI Developer, Trilogy (Remote) - $200,000/year USD',
    'description': "Crossover is the world's #1 source of full-time remote jobs. Our clients offer top-tier pay for
top-tier talent. We're recruiting this role for our client, Trilogy. Have you got what it takes?\nAr Trilogy, we 
believe the recent advances in generative AI are revolutionary once-a-decade disruptive changes, similar to how the
Internet and then cloud computing completely changed the software industry. We want to be on the cutting edge of 
these technology advancements when it comes to using generative AI in our development process, and we’re building a
new team that works this way.\nIf you are passionate about AI, excited to learn and experiment with the latest 
technologies for AI-assisted development, and want to be on the cutting edge of technologies in this space to 
become 10x more productive than a developer that doesn’t use AI, then you’re a great fit for this new 
team.\nBesides an exciting approach to how you do your day work, you’ll also get a chance to work on some really 
exciting products:\nCrossover - an AI-powered hiring platform that sifts through millions of applicants per year to
find the top 1% of global remote talent.gt.school - an online education platform that uses advanced data 
collection, algorithms, and AI-generated content to empower students to learn at their own pace.Data-driven Sales -
an initiative that leverages data and AI algorithms to streamline the sales process across a portfolio of over 100 
software products.\nWhat You Will Be Doing\nProposing and conducting experiments with generative AI to enhance 
engineering processes, including problem decomposition, solution design, code generation, and advanced 
troubleshooting.Documenting best practices and techniques in playbooks and guides that enable others to harness the
power of generative AI effectively.Using these AI tools and techniques to develop and ship actual features and 
products.\nWhat You Won’t Be Doing\nPlain-old regular development work.\nBasic Requirements\nAI Developer key 
responsibilities\nMinimum 5 years of experience in software development.Minimum 3 years as a primary technical 
contributor for a development team or software product.Extensive hands-on experience using generative AI for 
software engineering tasks or in a product you built.\nAbout Trilogy\nHundreds of software businesses run on the 
Trilogy Business Platform. For three decades, Trilogy has been known for 3 things: Relentlessly seeking top talent,
Innovating new technology, and incubating new businesses. Our technological innovation is spearheaded by a passion 
for simple customer-facing designs. Our incubation of new businesses ranges from entirely new moon-shot ideas to 
rearchitecting existing projects for today's modern cloud-based stack. Trilogy is a place where you can be 
surrounded with great people, be proud of doing great work, and grow your career by leaps and bounds.\nThere is so 
much to cover for this exciting role, and space here is limited. Hit the Apply button if you found this interesting
and want to learn more. We look forward to meeting you!\nWorking with Crossover\nThis is a full-time (40 hours per 
week), long-term position. The position is immediately available and requires entering into an independent 
contractor agreement with Crossover. The compensation level for this role is $100 USD/hour, which equates to 
$200,000 USD/year assuming 40 hours per week and 50 weeks per year. The payment period is weekly. Consult 
www.crossover.com/help-and-faqs for more details on this topic.\nWhat to expect next:\nYou will receive an email 
with a link to start your self-paced, online job application.Our hiring platform will guide you through a series of
online “screening” assessments to check for basic job fit, job-related skills, and finally a few real-world 
job-specific assignments.\nImportant! If you do not receive an email from us:\nFirst, emails may take up

200000.0

In [76]:
rich_print('''Crossover is the world's #1 source of full-time remote jobs. Our clients offer top-tier pay for
top-tier talent. We're recruiting this role for our client, Trilogy. Have you got what it takes?\nAr Trilogy, we 
believe the recent advances in generative AI are revolutionary once-a-decade disruptive changes, similar to how the
Internet and then cloud computing completely changed the software industry. We want to be on the cutting edge of 
these technology advancements when it comes to using generative AI in our development process, and we’re building a
new team that works this way.\nIf you are passionate about AI, excited to learn and experiment with the latest 
technologies for AI-assisted development, and want to be on the cutting edge of technologies in this space to 
become 10x more productive than a developer that doesn’t use AI, then you’re a great fit for this new 
team.\nBesides an exciting approach to how you do your day work, you’ll also get a chance to work on some really 
exciting products:\nCrossover - an AI-powered hiring platform that sifts through millions of applicants per year to
find the top 1% of global remote talent.gt.school - an online education platform that uses advanced data 
collection, algorithms, and AI-generated content to empower students to learn at their own pace.Data-driven Sales -
an initiative that leverages data and AI algorithms to streamline the sales process across a portfolio of over 100 
software products.\nWhat You Will Be Doing\nProposing and conducting experiments with generative AI to enhance 
engineering processes, including problem decomposition, solution design, code generation, and advanced 
troubleshooting.Documenting best practices and techniques in playbooks and guides that enable others to harness the
power of generative AI effectively.Using these AI tools and techniques to develop and ship actual features and 
products.\nWhat You Won’t Be Doing\nPlain-old regular development work.\nBasic Requirements\nAI Developer key 
responsibilities\nMinimum 5 years of experience in software development.Minimum 3 years as a primary technical 
contributor for a development team or software product.Extensive hands-on experience using generative AI for 
software engineering tasks or in a product you built.\nAbout Trilogy\nHundreds of software businesses run on the 
Trilogy Business Platform. For three decades, Trilogy has been known for 3 things: Relentlessly seeking top talent,
Innovating new technology, and incubating new businesses. Our technological innovation is spearheaded by a passion 
for simple customer-facing designs. Our incubation of new businesses ranges from entirely new moon-shot ideas to 
rearchitecting existing projects for today's modern cloud-based stack. Trilogy is a place where you can be 
surrounded with great people, be proud of doing great work, and grow your career by leaps and bounds.\nThere is so 
much to cover for this exciting role, and space here is limited. Hit the Apply button if you found this interesting
and want to learn more. We look forward to meeting you!\nWorking with Crossover\nThis is a full-time (40 hours per 
week), long-term position. The position is immediately available and requires entering into an independent 
contractor agreement with Crossover. The compensation level for this role is $100 USD/hour, which equates to 
$200,000 USD/year assuming 40 hours per week and 50 weeks per year. The payment period is weekly. Consult 
www.crossover.com/help-and-faqs for more details on this topic.\nWhat to expect next:\nYou will receive an email 
with a link to start your self-paced, online job application.Our hiring platform will guide you through a series of
online “screening” assessments to check for basic job fit, job-related skills, and finally a few real-world 
job-specific assignments.\nImportant! If you do not receive an email from us:\nFirst, emails may take up to 15 
minutes to send, refresh and check again.Second, check your spam and junk folders for an email from Crossover.com, 
mark as “Not Spam” since you will receive other emails as well.Third, we will send to whatever email account you 
indicated on the Apply form - by default, that is the email address you use as your LinkedIn username and it might 
be different than the one you have already checked.If all else fails, just reset your password by visiting 
https://www.crossover.com/auth/password-recovery if you already applied using LinkedIn EasyApply. Crossover Job 
Code: LJ-5106-US-NewYork-AIDeveloper\n''')

Crossover is the world's #1 source of full-time remote jobs. Our clients offer top-tier pay for
top-tier talent. We're recruiting this role for our client, Trilogy. Have you got what it takes?
Ar Trilogy, we 
believe the recent advances in generative AI are revolutionary once-a-decade disruptive changes, similar to how the
Internet and then cloud computing completely changed the software industry. We want to be on the cutting edge of 
these technology advancements when it comes to using generative AI in our development process, and we’re building a
new team that works this way.
If you are passionate about AI, excited to learn and experiment with the latest 
technologies for AI-assisted development, and want to be on the cutting edge of technologies in this space to 
become 10x more productive than a developer that doesn’t use AI, then you’re a great fit for this new 
team.
Besides an exciting approach to how you do your day work, you’ll also get a chance to work on some really 
exciting products:
Crossover - an AI-powered hiring platform that sifts through millions of applicants per year to
find the top 1% of global remote talent.gt.school - an online education platform that uses advanced data 
collection, algorithms, and AI-generated content to empower students to learn at their own pace.Data-driven Sales -
an initiative that leverages data and AI algorithms to streamline the sales process across a portfolio of over 100 
software products.
What You Will Be Doing
Proposing and conducting experiments with generative AI to enhance 
engineering processes, including problem decomposition, solution design, code generation, and advanced 
troubleshooting.Documenting best practices and techniques in playbooks and guides that enable others to harness the
power of generative AI effectively.Using these AI tools and techniques to develop and ship actual features and 
products.
What You Won’t Be Doing
Plain-old regular development work.
Basic Requirements
AI Developer key 
responsibilities
Minimum 5 years of experience in software development.Minimum 3 years as a primary technical 
contributor for a development team or software product.Extensive hands-on experience using generative AI for 
software engineering tasks or in a product you built.
About Trilogy
Hundreds of software businesses run on the 
Trilogy Business Platform. For three decades, Trilogy has been known for 3 things: Relentlessly seeking top talent,
Innovating new technology, and incubating new businesses. Our technological innovation is spearheaded by a passion 
for simple customer-facing designs. Our incubation of new businesses ranges from entirely new moon-shot ideas to 
rearchitecting existing projects for today's modern cloud-based stack. Trilogy is a place where you can be 
surrounded with great people, be proud of doing great work, and grow your career by leaps and bounds.
There is so 
much to cover for this exciting role, and space here is limited. Hit the Apply button if you found this interesting
and want to learn more. We look forward to meeting you!
Working with Crossover
This is a full-time (40 hours per 
week), long-term position. The position is immediately available and requires entering into an independent 
contractor agreement with Crossover. The compensation level for this role is $100 USD/hour, which equates to 
$200,000 USD/year assuming 40 hours per week and 50 weeks per year. The payment period is weekly. Consult 
www.crossover.com/help-and-faqs for more details on this topic.
What to expect next:
You will receive an email 
with a link to start your self-paced, online job application.Our hiring platform will guide you through a series of
online “screening” assessments to check for basic job fit, job-related skills, and finally a few real-world 
job-specific assignments.
Important! If you do not receive an email from us:
First, emails may take up to 15 
minutes to send, refresh and check again.Second, check your spam and junk folders for an email from C

In [74]:
# perform one prediction for sanity check

response = salary_prediction_agent.generate_response(
    messages=[
        Message(
            role='user',
            content=Content(
                text=JobPost(**x[test_set_ids[1]]).model_dump_json(),
            )
        )
    ],
    output_model=SalaryPrediction,
)

In [75]:
rich_print(
    SalaryPrediction.model_validate_json(
        response.content.text
    )
)

SalaryPrediction(
    reasons='Based on the job title, description, location, and experience level, the predicted salary for the job 
posting is $200,000 per year.',
    salary=200000
)

In [70]:
print(y[test_set_ids[2]])

208500.0


In [23]:

responses = await asyncio.gather(*[
    salary_prediction_agent.a_generate_response(
        messages=[
            Message(
                role='user',
                content=Content(
                    text=datum['title'],
                )
            )
        ],
        output_model=PricePrediction
    ) for datum in test_set
])

responses = [response for response in responses if response != []]

In [26]:
# Let's check how well the agent did
import numpy as np

def metric(predicted:List[PricePrediction], actual:List[PricePrediction]):
    if len(predicted) != len(actual):
        raise ValueError('predicted and actual must be the same length')
    
    value_map = {
        'positive': 1,
        'negative': -1,
        'neutral': 0
    }

    p = np.array([value_map[p.movement] for p in predicted])
    a = np.array([value_map[a.movement] for a in actual])
    
    error = np.square(p - a).mean()
    return error

metric(
    [PricePrediction.model_validate_json(response.content.text) for response in responses], 
    [PricePrediction(movement=datum.get('price_movement')) for datum in test_set]
)

1.14

An mean square error of 1.14 basically mean the model is doing nothing at predicting the sentiment. The actual movement is ever so slightly going opposite of what the model predicted!

In [27]:
from agentx.optimisers import TextualGradientPromptTrainer

reviewer = Agent(
    name='reviewer',
    generation_config=generation_config,
    system_prompt='You are a prompt engineer. Review the given prompt, error samples and give reasons why the prompt have gotten these examples wrong.',
)

def textual_gradient(
    prompt:str,
    input:List[str],
    predicted:List[PricePrediction],
    truth:List[PricePrediction],
) -> List[Message]:
    errors = [
        {
            'input': input,
            'predicted': predicted.movement,
            'truth': truth.movement,
        } for input, predicted, truth in zip(input, predicted, truth) if predicted != truth
    ]

    messages = [
        Message(
            role='user',
            content=Content(
                text='''Current prompt: {prompt}

Errors: {errors}'''.format(prompt=prompt, errors=errors, )
            ),
        )
    ]

    response = reviewer.generate_response(
        messages=messages
    )
    
    return messages + [response]

In [29]:
# test the textual_gradient
gradient = textual_gradient(
    prompt=sentiment_classification_agent.system_prompt,
    input=[datum['title'] for datum in test_set],
    predicted=[PricePrediction.model_validate_json(response.content.text) for response in responses],
    truth=[PricePrediction(movement=datum.get('price_movement')) for datum in test_set],
)

In [30]:
rich_print(gradient[-1].content.text)

The given prompt has several issues that have led to incorrect predictions. Here are the reasons why the prompt has
gotten these examples wrong:

1. Over-reliance on headline sentiment: The prompt solely relies on news headlines to predict the price movement of
QQQ. However, headlines can sometimes be misleading or not fully representative of the market sentiment. It is 
important to consider more comprehensive financial data and market analysis rather than just relying on news 
headlines.

2. Lack of context: News headlines alone may not provide sufficient context for predicting price movements 
accurately. Market movements are influenced by a wide range of factors including economic indicators, company 
performance, geopolitical events, and investor sentiment. Without considering these broader contextual factors, it 
is challenging to make accurate predictions solely based on news headlines.

3. Ambiguity in price movement definition: The prompt defines price movement as positive, negative, or neutral, 
without specifying the degree of movement that would be considered substantial. This lack of specificity can lead 
to subjective interpretations and inconsistent predictions.

4. Inconsistent correlation between news and price movement: The examples demonstrate instances where the predicted
price movement based on headlines does not align with the actual price movement. This inconsistency highlights the 
limitations of using news headlines as the sole basis for price predictions.

To improve the accuracy of predictions, the prompt should incorporate broader financial data, market analysis, and 
contextual factors, while also defining clear criteria for price movement predictions.

In [ ]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler